In [1]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import collections
import itertools
import time

import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import networkx as nx
import tensorflow as tf

from graph_nets import graphs
from graph_nets import utils_np
from graph_nets import utils_tf
from graph_nets.demos import models
from graph_nets import data_util

In [2]:
DISTANCE_WEIGHT_NAME = "distance"  # The name for the distance edge attribute.


def pairwise(iterable):
  """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
  a, b = itertools.tee(iterable)
  next(b, None)
  return zip(a, b)


def set_diff(seq0, seq1):
  """Return the set difference between 2 sequences as a list."""
  return list(set(seq0) - set(seq1))


def to_one_hot(indices, max_value, axis=-1):
  one_hot = np.eye(max_value)[indices]
  if axis not in (-1, one_hot.ndim):
    one_hot = np.moveaxis(one_hot, -1, axis)
  return one_hot


def get_node_dict(graph, attr):
  """Return a `dict` of node:attribute pairs from a graph."""
  return {k: v[attr] for k, v in graph.node.items()}


# Adapted from shortest_path demo
class ShortestPathGraphDataset(data_util.GraphDataset):
  def __init__(self, data_dir, min_nodes, max_nodes):
    features_list = [
      data_util.GraphFeature(key='input_graph',
                             node_feature_size=5,
                             edge_feature_size=1,
                             global_feature_size=1,
                             dtype='float32',
                             description='Graph to input to network'),
      data_util.GraphFeature(key='target_graph',
                             node_feature_size=2,
                             edge_feature_size=2,
                             global_feature_size=1,
                             dtype='float32',
                             description='Graph to output from network'),
      # Example of a non-graph feature
      data_util.TensorFeature(
          key='adj_mat_dense',
          shape=[max_nodes, max_nodes],
          dtype='float32',
          description='Sparse adjacency matrix of input graph'),
    ]
    super(ShortestPathGraphDataset, self).__init__(data_dir, features_list)
    self.min_nodes = min_nodes
    self.max_nodes = max_nodes

  def generate_graph(self,
                     rand,
                     num_nodes_min_max,
                     dimensions=2,
                     theta=1000.0,
                     rate=1.0):
    """Creates a connected graph.

    The graphs are geographic threshold graphs, but with added edges via a
    minimum spanning tree algorithm, to ensure all nodes are connected.

    Args:
      rand: A random seed for the graph generator. Default= None.
      num_nodes_min_max: A sequence [lower, upper) number of nodes per
        graph.
      dimensions: (optional) An `int` number of dimensions for the
        positions. Default= 2.
      theta: (optional) A `float` threshold parameters for the geographic
        threshold graph's threshold. Large values (1000+) make mostly trees.
        Try 20-60 for good non-trees. Default=1000.0.
      rate: (optional) A rate parameter for the node weight exponential
        sampling distribution. Default= 1.0.

    Returns:
      The graph.
    """
    # Sample num_nodes.
    num_nodes = rand.randint(*num_nodes_min_max)

    # Create geographic threshold graph.
    pos_array = rand.uniform(size=(num_nodes, dimensions))
    pos = dict(enumerate(pos_array))
    weight = dict(enumerate(rand.exponential(rate, size=num_nodes)))
    geo_graph = nx.geographical_threshold_graph(num_nodes,
                                                theta,
                                                pos=pos,
                                                weight=weight)

    # Create minimum spanning tree across geo_graph's nodes.
    distances = spatial.distance.squareform(
        spatial.distance.pdist(pos_array))
    i_, j_ = np.meshgrid(range(num_nodes), range(num_nodes), indexing="ij")
    weighted_edges = list(zip(i_.ravel(), j_.ravel(), distances.ravel()))
    mst_graph = nx.Graph()
    mst_graph.add_weighted_edges_from(weighted_edges,
                                      weight=DISTANCE_WEIGHT_NAME)
    mst_graph = nx.minimum_spanning_tree(mst_graph,
                                         weight=DISTANCE_WEIGHT_NAME)
    # Put geo_graph's node attributes into the mst_graph.
    for i in mst_graph.nodes():
        mst_graph.node[i].update(geo_graph.node[i])

    # Compose the graphs.
    combined_graph = nx.compose_all((mst_graph, geo_graph.copy()))
    # Put all distance weights into edge attributes.
    for i, j in combined_graph.edges():
      combined_graph.get_edge_data(i, j).setdefault(DISTANCE_WEIGHT_NAME,
                                                    distances[i, j])
    return combined_graph, mst_graph, geo_graph

  def add_shortest_path(self, rand, graph, min_length=1):
    """Samples a shortest path from A to B, adds attributes to indicate it.

    Args:
      rand: A random seed for the graph generator. Default= None.
      graph: A `nx.Graph`.
      min_length: (optional) An `int` minimum number of edges in the
        shortest path. Default= 1.

    Returns:
      The `nx.DiGraph` with the shortest path added.

    Raises:
      ValueError: All shortest paths are below the minimum length
    """
    # Map from node pairs to the length of their shortest path.
    pair_to_length_dict = {}
    try:
      # This is for compatibility with older networkx.
      lengths = nx.all_pairs_shortest_path_length(graph).items()
    except AttributeError:
      # This is for compatibility with newer networkx.
      lengths = list(nx.all_pairs_shortest_path_length(graph))
    for x, yy in lengths:
      for y, l in yy.items():
        if l >= min_length:
          pair_to_length_dict[x, y] = l
    if max(pair_to_length_dict.values()) < min_length:
      raise ValueError("All shortest paths are below the minimum length")
    # The node pairs which exceed the minimum length.
    node_pairs = list(pair_to_length_dict)

    # Computes probabilities per pair, to enforce uniform sampling of each
    # shortest path lengths.
    # The counts of pairs per length.
    counts = collections.Counter(pair_to_length_dict.values())
    prob_per_length = 1.0 / len(counts)
    probabilities = [
        prob_per_length / counts[pair_to_length_dict[x]]
        for x in node_pairs
    ]

    # Choose the start and end points.
    i = rand.choice(len(node_pairs), p=probabilities)
    start, end = node_pairs[i]
    path = nx.shortest_path(graph,
                            source=start,
                            target=end,
                            weight=DISTANCE_WEIGHT_NAME)

    # Creates a directed graph, to store directed path from start to end.
    digraph = graph.to_directed()

    # Add "start", "end", and "solution" attributes to the nodes and edges.
    digraph.add_node(start, start=True)
    digraph.add_node(end, end=True)
    digraph.add_nodes_from(set_diff(digraph.nodes(), [start]), start=False)
    digraph.add_nodes_from(set_diff(digraph.nodes(), [end]), end=False)
    digraph.add_nodes_from(set_diff(digraph.nodes(), path), solution=False)
    digraph.add_nodes_from(path, solution=True)
    path_edges = list(pairwise(path))
    digraph.add_edges_from(set_diff(digraph.edges(), path_edges),
                           solution=False)
    digraph.add_edges_from(path_edges, solution=True)

    return digraph

  def graph_to_input_target(self, graph):
    """Returns 2 graphs with input and target feature vectors for training.

    Args:
      graph: An `nx.DiGraph` instance.

    Returns:
      The input `nx.DiGraph` instance.
      The target `nx.DiGraph` instance.

    Raises:
      ValueError: unknown node type
    """

    def create_feature(attr, fields):
      return np.hstack([np.array(attr[field], dtype=float) for field in fields])

    input_node_fields = ("pos", "weight", "start", "end")
    input_edge_fields = ("distance", )
    target_node_fields = ("solution", )
    target_edge_fields = ("solution", )

    input_graph = graph.copy()
    target_graph = graph.copy()

    solution_length = 0
    for node_index, node_feature in graph.nodes(data=True):
      input_graph.add_node(node_index,
                           features=create_feature(
                               node_feature, input_node_fields))
      target_node = to_one_hot(
          create_feature(node_feature, target_node_fields).astype(int),
          2)[0]
      target_graph.add_node(node_index, features=target_node)
      solution_length += int(node_feature["solution"])
    solution_length /= graph.number_of_nodes()

    for receiver, sender, features in graph.edges(data=True):
      input_graph.add_edge(sender,
                           receiver,
                           features=create_feature(
                               features, input_edge_fields))
      target_edge = to_one_hot(
          create_feature(features, target_edge_fields).astype(int), 2)[0]
      target_graph.add_edge(sender, receiver, features=target_edge)

    input_graph.graph["features"] = np.array([0.0])
    target_graph.graph["features"] = np.array([solution_length],
                                              dtype=float)

    return input_graph, target_graph

  def gen_sample(self, name, index):
    seed = abs(hash(name)*index) % (2**32)
    theta = 1001.0
    rate = 1.1
    rand = np.random.RandomState(seed=seed)  # TODO: Figure out seed
    graph = self.generate_graph(rand, (self.min_nodes, self.max_nodes),
                                theta=theta)[0]
    graph = self.add_shortest_path(rand, graph)
    input_graph, target_graph = self.graph_to_input_target(graph)
    # import pdb; pdb.set_trace()
    input_graph_dict = utils_np.networkx_to_data_dict(input_graph)
    target_graph_dict = utils_np.networkx_to_data_dict(target_graph)
    adj_mat_dense = np.zeros((self.max_nodes, self.max_nodes))
    n_node = input_graph_dict['n_node']
    adj_mat_dense[:n_node, :n_node] = nx.to_numpy_matrix(input_graph)
    adj_mat_sparse = data_util.np_dense_to_sparse(adj_mat_dense)
    return {
        'input_graph': input_graph_dict,
        'target_graph': target_graph_dict,
        'adj_mat_dense': adj_mat_dense,
        'adj_mat_sparse': adj_mat_sparse,
    }


In [3]:
# GENERATE DATASET
print("Generating Pose Graphs")
testgraphs_data_dir = 'testgraphs'
if not os.path.exists(testgraphs_data_dir):
  os.makedirs(testgraphs_data_dir)
mydataset = ShortestPathGraphDataset(testgraphs_data_dir, 30, 40)

types = [('train', 100), ('test', 100)]
for t, n in types:
  dname = os.path.join(testgraphs_data_dir, t)
  if not os.path.exists(dname):
    os.makedirs(dname)
  mydataset.convert_dataset(t, n)

# Generate numpy test
mydataset.create_np_dataset('np_train', 100)
mydataset.create_np_dataset('np_test', 100)


  6%|▌         | 6/100 [00:00<00:01, 55.49it/s]

Generating Pose Graphs
Writing dataset to testgraphs/train


  6%|▌         | 6/100 [00:00<00:01, 58.69it/s]

Writing dataset to testgraphs/test


  6%|▌         | 6/100 [00:00<00:01, 55.07it/s]

Writing dataset to testgraphs/np_train


  5%|▌         | 5/100 [00:00<00:01, 47.84it/s]

Writing dataset to testgraphs/np_test


100%|██████████| 100/100 [00:01<00:00, 51.07it/s]


In [4]:
def compute_accuracy(target, output, use_nodes=True, use_edges=False):
  """Calculate model accuracy.

  Returns the number of correctly predicted shortest path nodes and the number
  of completely solved graphs (100% correct predictions).

  Args:
    target: A `graphs.GraphsTuple` that contains the target graph.
    output: A `graphs.GraphsTuple` that contains the output graph.
    use_nodes: A `bool` indicator of whether to compute node accuracy or not.
    use_edges: A `bool` indicator of whether to compute edge accuracy or not.

  Returns:
    correct: A `float` fraction of correctly labeled nodes/edges.
    solved: A `float` fraction of graphs that are completely correctly
      labeled.

  Raises:
    ValueError: Nodes or edges (or both) must be used
  """
  if not use_nodes and not use_edges:
      raise ValueError("Nodes or edges (or both) must be used")
  tdds = utils_np.graphs_tuple_to_data_dicts(target)
  odds = utils_np.graphs_tuple_to_data_dicts(output)
  cs = []
  ss = []
  for td, od in zip(tdds, odds):
    xn = np.argmax(td["nodes"], axis=-1)
    yn = np.argmax(od["nodes"], axis=-1)
    xe = np.argmax(td["edges"], axis=-1)
    ye = np.argmax(od["edges"], axis=-1)
    c = []
    if use_nodes:
      c.append(xn == yn)
    if use_edges:
      c.append(xe == ye)
    c = np.concatenate(c, axis=0)
    s = np.all(c)
    cs.append(c)
    ss.append(s)
  correct = np.mean(np.concatenate(cs, axis=0))
  solved = np.mean(np.stack(ss))
  return correct, solved


def create_loss_ops(target_op, output_ops):
  loss_ops = [
      tf.losses.softmax_cross_entropy(target_op.nodes, output_op.nodes) +
      tf.losses.softmax_cross_entropy(target_op.edges, output_op.edges)
      for output_op in output_ops
  ]
  return loss_ops


In [5]:
def ret_none(x):
  return None

def run_training(input_ph,
                 target_ph,
                 get_vals=ret_none,
                 get_feed_dict=ret_none):
  # Connect the data to the model.
  # Instantiate the model.
  model = models.EncodeProcessDecode(edge_output_size=2, node_output_size=2)
  # A list of outputs, one per processing step.
  num_processing_steps = 10
  output_ops_tr = model(input_ph, num_processing_steps)
  output_ops_ge = model(input_ph, num_processing_steps)
  # Training loss.
  loss_ops_tr = create_loss_ops(target_ph, output_ops_tr)
  # Loss across processing steps.
  loss_op_tr = sum(loss_ops_tr) / num_processing_steps
  # Test/generalization loss.
  loss_ops_ge = create_loss_ops(target_ph, output_ops_ge)
  loss_op_ge = loss_ops_ge[-1]  # Loss from final processing step.

  # Optimizer.
  learning_rate = 1e-3
  optimizer = tf.train.AdamOptimizer(learning_rate)
  step_op = optimizer.minimize(loss_op_tr)

  try:
      sess.close()
  except NameError:
      pass
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())

  last_iteration = 0
  logged_iterations = []
  losses_tr = []
  corrects_tr = []
  solveds_tr = []
  losses_ge = []
  corrects_ge = []
  solveds_ge = []

  # How much time between logging and printing the current results.
  num_training_iterations = 1000
  log_every_seconds = 2

  print("# (iteration number), T (elapsed seconds), "
        "Ltr (training loss), Lge (test/generalization loss), "
        "Ctr (training fraction nodes/edges labeled correctly), "
        "Str (training fraction examples solved correctly), "
        "Cge (test/generalization fraction nodes/edges labeled correctly), "
        "Sge (test/generalization fraction examples solved correctly)")

  start_time = time.time()
  last_log_time = start_time
  for iteration in range(last_iteration, num_training_iterations):
    last_iteration = iteration
    vals = get_vals(iteration)
    train_values = sess.run({
        "step": step_op,
        "target": target_ph,
        "loss": loss_op_tr,
        "outputs": output_ops_tr
    }, feed_dict=get_feed_dict(vals))
    the_time = time.time()
    elapsed_since_last_log = the_time - last_log_time
    if elapsed_since_last_log > log_every_seconds:
      last_log_time = the_time
      test_values = sess.run({
          "target": target_ph,
          "loss": loss_op_ge,
          "outputs": output_ops_ge
      }, feed_dict=get_feed_dict(vals))
      correct_tr, solved_tr = compute_accuracy(train_values["target"],
                                               train_values["outputs"][-1],
                                               use_edges=True)
      correct_ge, solved_ge = compute_accuracy(test_values["target"],
                                               test_values["outputs"][-1],
                                               use_edges=True)
      elapsed = time.time() - start_time
      losses_tr.append(train_values["loss"])
      corrects_tr.append(correct_tr)
      solveds_tr.append(solved_tr)
      losses_ge.append(test_values["loss"])
      corrects_ge.append(correct_ge)
      solveds_ge.append(solved_ge)
      logged_iterations.append(iteration)
      print("# {:05d}, T {:.1f}, Ltr {:.4f}, Lge {:.4f}, Ctr {:.4f}, Str"
            " {:.4f}, Cge {:.4f}, Sge {:.4f}".format(iteration, elapsed,
                                                     train_values["loss"],
                                                     test_values["loss"],
                                                     correct_tr, solved_tr,
                                                     correct_ge, solved_ge))
    

In [6]:
# Create tensors
print('#####################################################')
print("Using TFRecords....")
sample = mydataset.load_batch('train', 16)
input_ph = sample['input_graph']
target_ph = sample['target_graph']
run_training(input_ph,
             target_ph,
             get_vals=ret_none,
             get_feed_dict=ret_none)


/home/stephen/tensorflow/lib/python3.5/site-packages/tensorflow/python/client/session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


#####################################################
Using TFRecords....


/home/stephen/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# (iteration number), T (elapsed seconds), Ltr (training loss), Lge (test/generalization loss), Ctr (training fraction nodes/edges labeled correctly), Str (training fraction examples solved correctly), Cge (test/generalization fraction nodes/edges labeled correctly), Sge (test/generalization fraction examples solved correctly)
# 00000, T 9.2, Ltr 2.6663, Lge 2.1821, Ctr 0.4958, Str 0.0000, Cge 0.5713, Sge 0.0000
# 00021, T 10.1, Ltr 1.0943, Lge 1.0865, Ctr 0.7840, Str 0.0000, Cge 0.7794, Sge 0.0000
# 00067, T 12.2, Ltr 0.9338, Lge 0.9289, Ctr 0.8147, Str 0.0000, Cge 0.8131, Sge 0.0000
# 00112, T 14.2, Ltr 0.9161, Lge 0.7723, Ctr 0.8155, Str 0.0000, Cge 0.8659, Sge 0.0000
# 00158, T 16.2, Ltr 0.9370, Lge 0.8810, Ctr 0.7917, Str 0.0000, Cge 0.8062, Sge 0.0000
# 00204, T 18.3, Ltr 0.7571, Lge 0.7763, Ctr 0.8652, Str 0.0000, Cge 0.8611, Sge 0.0000
# 00251, T 20.3, Ltr 0.9121, Lge 0.9017, Ctr 0.7981, Str 0.0000, Cge 0.7999, Sge 0.0000
# 00298, T 22.3, Ltr 0.8320, Lge 0.7549, Ctr 0.8094, Str

In [7]:
# Create tensors
print('#####################################################')
print("Using Placeholders....")
sample, placeholders = mydataset.get_placeholders(True)
input_ph = sample['input_graph']
target_ph = sample['target_graph']

def get_vals(iteration):
  index = iteration % mydataset.sizes['np_train']
  return mydataset.load_npz_file('np_train', index)

def get_feed_dict(vals):
  fdict = mydataset.get_feed_dict(placeholders, vals, True)
  return fdict

run_training(input_ph,
             target_ph,
             get_vals=get_vals,
             get_feed_dict=get_feed_dict)

#####################################################
Using Placeholders....
# (iteration number), T (elapsed seconds), Ltr (training loss), Lge (test/generalization loss), Ctr (training fraction nodes/edges labeled correctly), Str (training fraction examples solved correctly), Cge (test/generalization fraction nodes/edges labeled correctly), Sge (test/generalization fraction examples solved correctly)
# 00000, T 9.9, Ltr 0.9613, Lge 0.7695, Ctr 0.8447, Str 0.0000, Cge 0.8932, Sge 0.0000
# 00050, T 10.7, Ltr 0.7273, Lge 0.7298, Ctr 0.9340, Str 0.0000, Cge 0.9340, Sge 0.0000
# 00172, T 12.7, Ltr 1.2060, Lge 1.1889, Ctr 0.7010, Str 0.0000, Cge 0.6907, Sge 0.0000
# 00290, T 14.8, Ltr 0.6888, Lge 0.6292, Ctr 0.9072, Str 0.0000, Cge 0.9175, Sge 0.0000
# 00409, T 16.8, Ltr 0.9671, Lge 0.9434, Ctr 0.7500, Str 0.0000, Cge 0.7500, Sge 0.0000
# 00528, T 18.8, Ltr 0.9464, Lge 0.9450, Ctr 0.8191, Str 0.0000, Cge 0.8191, Sge 0.0000
# 00647, T 20.8, Ltr 0.8606, Lge 0.8613, Ctr 0.8214, Str 0.0000, Cg